In [13]:
from sklearn.datasets import load_iris
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch

# load iris dataset
x, t = load_iris(return_X_y=True)

# transform to the format for learning with PyTorch
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

# transform input value and target value together to a object "dataset"
dataset = TensorDataset(x, t)

# set sample num of each dataset
# train : val : test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

# because of random split, fix random number seed to maintain reproductivity
torch.manual_seed(0)

# split dataset
train, val, test = random_split(dataset, [n_train, n_val, n_test])

# definition of batch size
batch_size = 32

# preparing DataLoader
# shuffle is False in default, so only train dataset is set True
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [14]:
import ax

In [15]:
parameters = [
    {'name': 'x1', 'type': 'range', 'bounds': [-10., 10.]},
    {"name": 'x2', 'type': 'range', 'bounds': [-10., 10.]}
]

In [16]:
def evaluation_function(parameters):
    x1 = parameters.get('x1')
    x2 = parameters.get('x2')
    f = x1**2 + x2**2
    return f

In [17]:
# optimize
# minimize==True: minimize, minimize==False: maximize
# because of using random number seed, fixing seed is required
results = ax.optimize(parameters, evaluation_function, minimize=True, random_seed=0)

AttributeError: module 'ax' has no attribute 'optimize'

In [65]:
# check result
best_parameters, values, experiment, model = results

# parameters after optimization
best_parameters

{'x1': 0.000986274809164911, 'x2': -0.13807739501201866}

In [66]:
# value of function with parameters after optimization
values

({'objective': 0.02335578038955788},
 {'objective': {'objective': 0.047031639600553295}})

In [67]:
import pytorch_lightning as pl
import torchmetrics
import torch.nn.functional as F
import torch.nn as nn
import torch

class Net(pl.LightningModule):
    def __init__(self, n_mid=4, lr=0.01):
        super().__init__()
        self.fc1 = nn.Linear(4, n_mid)
        self.fc2 = nn.Linear(n_mid, 3)
        self.lr = lr

        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()
        self.test_acc = torchmetrics.Accuracy()

    def forward(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.fc2(h)
        return h

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('train_liss', loss, on_step=True, on_epoch=True)
        self.log('train_acc', self.train_acc(y, t), on_step=True, on_epoch=True)
        return loss
    
    # process about val data
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', self.val_acc(y, t), on_step=False, on_epoch=True)
        return loss

    # process about test data
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_acc', self.test_acc(y, t), on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.lr)
        return optimizer

In [68]:
pl.seed_everything(0)
net = Net()
trainer = pl.Trainer(max_epochs=10)
trainer.fit(net, train_loader, val_loader)

Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type     | Params
---------------------------------------
0 | fc1       | Linear   | 20    
1 | fc2       | Linear   | 15    
2 | train_acc | Accuracy | 0     
3 | val_acc   | Accuracy | 0     
4 | test_acc  | Accuracy | 0     
---------------------------------------
35        Trainable params
0         Non-trainable params
35        Total params
0.000     Total estimated model params size (MB)


c:\Users\nagas\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\nagas\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:219: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\nagas\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\trainer.py:1894: PossibleUserWarning: The number of train

Epoch 9: 100%|██████████| 3/3 [00:00<00:00, 99.99it/s, loss=1.05, v_num=7] 

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 3/3 [00:00<00:00, 83.33it/s, loss=1.05, v_num=7]


In [72]:
from sklearn.datasets import load_iris
from torch.utils.data import DataLoader, TensorDataset, random_split

# load iris dataset
x, t = load_iris(return_X_y=True)

# transform to the format for learning with PyTorch
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

# transform input value and target value together to a object "dataset"
dataset = TensorDataset(x, t)

# set sample num of each dataset
# train : val : test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

# because of random split, fix random number seed to maintain reproductivity
torch.manual_seed(0)

# split dataset
train, val, test = random_split(dataset, [n_train, n_val, n_test])

# definition of batch size
batch_size = 32

# preparing DataLoader
# shuffle is False in default, so only train dataset is set True
train_dataloader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [73]:
# val_acc of last epoch
trainer.callback_metrics['val_acc']

tensor(0.7000)

In [74]:
parameters = [
    {'name': 'n_mid', 'type': 'range', 'bounds': [1, 100]},\
    {'name': 'lr', 'type': 'range', 'bounds': [0.001, 0.1]}
]

def evaluation_function(parameters):

    # get parameters
    n_mid = parameters.get('n_mid')
    lr = parameters.get('lr')

    # definition of network and learning
    torch.manual_seed(0)
    net = Net(n_mid=n_mid, lr=lr)
    trainer = pl.Trainer()
    trainer.fit(net)

    # return val_acc as target function value
    val_acc = trainer.callback_metrics['val_acc']

    # calculate test_acc for checking
    trainer.test()
    test_acc = trainer.callback_metrics['test_acc']

    # show selected hyper parameters of each trial
    print('n_mid: ', n_mid)
    print('lr: ', lr)
    print('val_acc: ', val_acc)
    print('test_acc: ', test_acc)

    return val_acc

# optimization
# default: minimize=False
results = ax.optimize(parameters, evaluation_function, random_seed=0)

# get result
best_parameters, values, experiment, best_model = results

# optimized parameters
best_parameters

[INFO 08-26 23:03:19] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter n_mid. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-26 23:03:19] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-26 23:03:19] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='n_mid', parameter_type=INT, range=[1, 100]), RangeParameter(name='lr', parameter_type=FLOAT, range=[0.001, 0.1])], parameter_constraints=[]).
[INFO 08-26 23:03:19] ax.modelbridge.dispatch_utils: Using Bayesian optimization since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 08-26 23:03:19] ax.modelbridge.dispatch_utils: 

ValueError: Cannot identify best point if experiment contains no data.